In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local').appName('preprocessing').getOrCreate()

In [3]:
bank_full = spark.read.csv('bank-full.csv',sep = ";",header=True,inferSchema=True)

In [4]:
bank_full_copy  = bank_full.select("*")

In [5]:
bank_full_copy.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [6]:
bank_full.count()

45211

In [7]:
bank_full.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

from pyspark.sql.functions import col,lit, when
month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
new_data = bank_full.withColumn("year", 2008)
new_data = new_data.withColumn("year", when(col("month") == month_lst[0], 2008).otherwise(col("year")))
new_data.show()

In [ ]:
def year_mapper(data, start_yr,end_yr):
    from pyspark.sql.functions import when, col, monotonically_increasing_id
    from pyspark.sql.window import Window
    from pyspark.sql.functions import lag
    month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

    # Make a copy of the original dataframe
    new_data = bank_full.select("*")

    # Insert a new "year" column filled with zeros
    new_data = new_data.withColumn("year", lit(0))
    
    # Select columns and move year column to desired index
    columns = new_data.columns
    columns.remove("year")
    columns.insert(10, "year") # 10 is index
    new_data = new_data.select(*columns)

    # Set the first year to the start year
    current_year = int(start_yr)
    new_data = new_data.withColumn("row_index", monotonically_increasing_id())
    
    # set the value of the first row's year column to 2050
    new_data = new_data.withColumn("year", when(col("row_index") == 0, current_year).otherwise(col("year")))
    
    for i in range(1,len(new_data.collect()[1:])):
        # If the current month is earlier in the year than the previous month, increment the year
        if month_lst.index(new_data.filter(new_data.row_index == i).collect()[0]["month"]) < month_lst.index(new_data.filter(new_data.row_index == i-1).collect()[0]["month"]):
            current_year += 1

        new_data = new_data.withColumn("year", when(col("row_index") == i, current_year).otherwise(col("year")))

        # If the current year exceeds the end year, break out of the loop
        if current_year > end_yr:
            break
            
    return new_data.drop("row_index")

In [ ]:
bank_full_yr.select(["month"])

In [ ]:
month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

month_lst.index(bank_full_yr.filter(bank_full_yr.row_index == 1).collect()[0]["month"])

In [ ]:
%%time
from pyspark.sql.functions import when, col, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

start_yr = 2008
end_yr = 2010
month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# Make a copy of the original dataframe
new_data = bank_full.select("*")

# Insert a new "year" column filled with zeros
new_data = new_data.withColumn("year", lit(0))
    
# Select columns and move year column to desired index
columns = new_data.columns
columns.remove("year")
columns.insert(10, "year") # 10 is index
new_data = new_data.select(*columns)

# Set the first year to the start year
current_year = int(start_yr)
new_data = new_data.withColumn("row_index", monotonically_increasing_id())
    
# set the value of the first row's year column to 2050
new_data = new_data.withColumn("year", when(col("row_index") == 0, current_year).otherwise(col("year")))
    
#for i in range(1,len(new_data.collect()[1:])):
for i in range(1, 50): 
    # If the current month is earlier in the year than the previous month, increment the year
    if month_lst.index(new_data.filter(new_data.row_index == i).collect()[0]["month"]) < month_lst.index(new_data.filter(new_data.row_index == i-1).collect()[0]["month"]):
        current_year += 1

    new_data = new_data.withColumn("year", when(col("row_index") == i, current_year).otherwise(col("year")))

    # If the current year exceeds the end year, break out of the loop
    if current_year > end_yr:
        break
    

In [ ]:
%%time
new_data.show()

In [ ]:
new_data.filter( new_data.row_index == 99 ).collect()

In [ ]:
%%time
bank_full_yr = year_mapper(data = bank_full, start_yr=2008,end_yr=2010)
bank_full_yr.show()

In [ ]:

print(new_data.dropDuplicates(["year"]).select("year").collect())

In [ ]:

bank_full_yr.write.options(header='True', delimiter=',') \
 .csv("D:/Lemonrice/virtual_demo/bank_full_yr/")

In [ ]:
#For checking the 
bank_pdf = bank_full_yr.toPandas()

In [ ]:
bank_pdf.to_csv("bank_pdf_6thMarch.csv")